In [6]:
# 코랩에서 안정적으로 설치되는 형태소 분석기 찾아서 삼만리! - ETRI OPEN API, Kkma, Kiwi, KHAIII -> OKT.
# 참내! 인공지능이 고집부린다고 뭐라고 하네요.. 결국 OKT로 실험 ㅜㅜ
# ================================================================================
# ✅ Cell 1: 라이브러리 임포트 및 설치
# ================================================================================
# 라이브러리 설치 및 임포트
import subprocess
import sys
import os

print("=" * 80)
print("📦 라이브러리 설치 및 임포트")
print("=" * 80)

# Step 0: OpenJDK 설치 및 JAVA_HOME 설정 (konlpy 요구사항)
print("0️⃣ OpenJDK 설치 및 JAVA_HOME 설정...")
# 기존 openjdk-install 대신 apt-get을 사용합니다.
subprocess.check_call(['apt-get', 'update'])
subprocess.check_call(['apt-get', 'install', '-y', 'openjdk-11-jdk-headless'])
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-11-openjdk-amd64"
print(f"   ✅ JAVA_HOME 설정 완료: {os.environ['JAVA_HOME']}")

# Step 1: 필수 패키지 설치
print("\n1️⃣ 필수 패키지 설치...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "konlpy", "sentencepiece"])
print("   ✅ 완료")

# Step 2: 라이브러리 임포트
print("\n2️⃣ 라이브러리 임포트...")
import torch
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from konlpy.tag import Okt  # ⭐ OKT 사용
from collections import Counter
from tqdm import tqdm
import sentencepiece as spm
import re

print("   ✅ 완료")

# Step 3: OKT 초기화
print("\n3️⃣ OKT 형태소 분석기 초기화...")
okt = Okt()
print("   ✅ OKT 로드 완료")

# Step 4: 버전 확인
print("\n4️⃣ 버전 확인...")
print(f"   PyTorch: {torch.__version__}")
print(f"   NumPy: {np.__version__}")
print(f"   Pandas: {pd.__version__}")

# Step 5: OKT 테스트
print("\n5️⃣ OKT 동작 테스트...")
test_result = okt.morphs("이 영화 정말 재미있었어")
print(f"   ✅ OKT 정상 작동")
print(f"   분석 결과: {test_result}")

print("\n" + "=" * 80)
print("🎉 환경셋팅 완료!")
print("=" * 80)


📦 라이브러리 설치 및 임포트
0️⃣ OpenJDK 설치 및 JAVA_HOME 설정...
   ✅ JAVA_HOME 설정 완료: /usr/lib/jvm/java-11-openjdk-amd64

1️⃣ 필수 패키지 설치...
   ✅ 완료

2️⃣ 라이브러리 임포트...
   ✅ 완료

3️⃣ OKT 형태소 분석기 초기화...
   ✅ OKT 로드 완료

4️⃣ 버전 확인...
   PyTorch: 2.8.0+cu126
   NumPy: 2.0.2
   Pandas: 2.2.2

5️⃣ OKT 동작 테스트...
   ✅ OKT 정상 작동
   분석 결과: ['이', '영화', '정말', '재미있었어']

🎉 환경셋팅 완료!


데이터 업로딩! 수동으로! 다음 코드블럭 실행하기 전에!!

In [7]:
# ================================================================================
# ✅ Cell 2: 데이터 로드
# ================================================================================
train = pd.read_table("/content/ratings_train.txt")
test = pd.read_table("/content/ratings_test.txt")

print("✅ 데이터 로드 완료")
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print("\n전처리 결과 샘플:")
print(train.head())


# ================================================================================
# ✅ Cell 3: 텍스트 전처리
# ================================================================================
# 불용어 리스트
STOPWORDS = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# 맞춤법 변형 사전
SPELLING_DICT = {
    '굳': ['궅', '굳', '굿'],
    '미쳤': ['미첫', '미쳣', '미첬', '미쳤', 'ㅁㅊ'],
    '괜찮': ['괜찮', '괜춘', '괜찬', 'ㄱㅊ', '갠찬', '갠찮', '괸찬', '괸찮'],
    '봤': ['봣'],
    '겠': ['겟']
}

TEXT_COL = "document"

def preprocess_text(text):
    """텍스트 전처리 함수"""
    if pd.isna(text):
        return ""

    text = str(text)

    # 1. 반복되는 문장 부호 제거 (2개 이상 → 1개)
    text = re.sub(r'([.!?…;])\1+', r'\1', text)

    # 2. 맞춤법 변형 통일
    for correct, variations in SPELLING_DICT.items():
        for variant in variations:
            text = text.replace(variant, correct)

    # 3. 반복 문자 제거 (3번 이상 반복 → 2번)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)

    # 4. 자음/모음 단독 제거
    text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+', ' ', text)

    # 5. 영어/숫자/한글/문장부호만 남기기
    text = re.sub(r'[^가-힣a-zA-Z0-9\s.!?,]', ' ', text)

    # 6. 문장부호 앞뒤로 공백 추가
    text = re.sub(r'([.!?,])', r' \1 ', text)

    # 7. 영어 소문자 변환
    text = text.lower()

    # 8. 불용어 제거
    words = text.split()
    filtered_words = []
    for word in words:
        if word in STOPWORDS:
            continue
        found = False
        for stopword in sorted(STOPWORDS, key=len, reverse=True):
            if len(word) > len(stopword) and word.endswith(stopword):
                cleaned = word[:-len(stopword)]
                if len(cleaned) >= 2:
                    filtered_words.append(cleaned)
                    found = True
                    break
        if not found:
            filtered_words.append(word)

    text = ' '.join(filtered_words)

    # 9. 연속 공백을 하나로
    text = re.sub(r'\s+', ' ', text)

    # 10. 앞뒤 공백 제거
    return text.strip()


def preprocess_dataframe(df, text_col=TEXT_COL):
    """데이터프레임 전처리 함수"""
    print(f"전처리 전 데이터 크기: {len(df)}")

    # 1. 결측치 제거
    df = df.dropna(subset=[text_col])
    print(f"결측치 제거 후: {len(df)}")

    # 2. 텍스트 전처리 적용
    df[text_col] = df[text_col].apply(preprocess_text)

    # 3. 빈 문자열 제거
    df = df[df[text_col].str.strip() != '']
    print(f"빈 문자열 제거 후: {len(df)}")

    # 4. 중복 행 제거
    df = df.drop_duplicates(subset=[text_col])
    print(f"중복 제거 후: {len(df)}")

    return df.reset_index(drop=True)


print("=" * 80)
print("데이터 전처리 시작")
print("=" * 80)

print("\n📊 Train 데이터 전처리:")
train_processed = preprocess_dataframe(train.copy())

print("\n📊 Test 데이터 전처리:")
test_processed = preprocess_dataframe(test.copy())

print(f"\n✅ 전처리 완료!")
print(f"Train 데이터: {len(train_processed)}개")
print(f"Test 데이터: {len(test_processed)}개")
print("\n전처리 결과 샘플:")
print(train_processed.head(10))


# ================================================================================
# ✅ Cell 4: SentencePiece 모델 학습 (OKT 기반)
# ================================================================================
print("\n" + "=" * 80)
print("SentencePiece 모델 학습 (OKT 기반)")
print("=" * 80)

# Step 1: OKT로 형태소 분석
print("\n🔄 Step 1: OKT 형태소 분석 진행 중...")
corpus = train_processed['document'].tolist()

# 형태소 분석된 텍스트를 임시 파일에 저장
temp_corpus_path = '/tmp/morphs_corpus_okt.txt'

with open(temp_corpus_path, 'w', encoding='utf-8') as f:
    for text in tqdm(corpus, desc="OKT morphs"):
        # ⭐ OKT 형태소 분석
        morphs = okt.morphs(text)
        f.write(' '.join(morphs) + '\n')

print(f"✅ 형태소 분석 완료: {temp_corpus_path}")

# Step 2: SentencePiece 모델 학습
print("\n🔄 Step 2: SentencePiece 모델 학습 중...")

sp_model_path = '/tmp/sentencepiece_okt'
vocab_size = 8000

spm.SentencePieceTrainer.train(
    input=temp_corpus_path,
    model_prefix=sp_model_path,
    vocab_size=vocab_size,
    character_coverage=1.0,
    model_type='unigram',
    unk_id=0,
    bos_id=1,
    eos_id=2,
    pad_id=3,
    normalization_rule_name='identity'
)

print(f"✅ SentencePiece 모델 학습 완료")
print(f"   Model: {sp_model_path}.model")
print(f"   Vocab size: {vocab_size:,}")

# Step 3: SentencePiece 모델 로드
sp = spm.SentencePieceProcessor()
sp.Load(f'{sp_model_path}.model')

print(f"\n✅ SentencePiece 모델 로드 완료")
print(f"   Vocab size: {sp.vocab_size()}")


# ================================================================================
# ✅ Cell 5: OKT + SentencePiece 토크나이제이션 테스트
# ================================================================================
print("\n" + "=" * 80)
print("OKT + SentencePiece 토크나이제이션 테스트")
print("=" * 80)

test_texts = [
    "이 영화 정말 재미있었어",
    "최악의 영화였다",
    "배우들의 연기가 훌륭했다"
]

for text in test_texts:
    # ⭐ OKT 형태소 분석
    morphs = okt.morphs(text)
    morphs_text = ' '.join(morphs)

    # SentencePiece 인코딩
    encoded = sp.EncodeAsIds(morphs_text)

    # 디코딩 확인
    decoded = sp.DecodeIds(encoded)

    print(f"\n원본: {text}")
    print(f"형태소: {morphs_text}")
    print(f"인코딩: {encoded}")
    print(f"디코딩: {decoded}")


# ================================================================================
# ✅ Cell 6: 토크나이징 함수
# ================================================================================

def okt_sentencepiece_tokenize(okt, sp, corpus, max_length=130):
    """
    OKT 형태소 분석 + SentencePiece 토크나이제이션

    Args:
        okt: OKT 토크나이저 객체
        sp: SentencePiece 프로세서 객체
        corpus: 전처리된 텍스트 리스트
        max_length: 최대 시퀀스 길이

    Returns:
        tensor: 패딩된 토큰 ID 텐서
        vocab_size: SentencePiece 어휘집 크기
    """
    # Step 1: OKT 형태소 분석
    tokenized_texts = []
    all_token_ids = []

    print("🔄 OKT 형태소 분석 + SentencePiece 토크나이제이션 진행 중...")
    for text in tqdm(corpus, desc="OKT + SentencePiece"):
        # ⭐ OKT로 형태소 분석
        morphs = okt.morphs(text)
        morphs_text = ' '.join(morphs)

        # SentencePiece로 서브워드 토크나이제이션
        token_ids = sp.EncodeAsIds(morphs_text)

        # 최대 길이로 자르기
        if len(token_ids) > max_length:
            token_ids = token_ids[:max_length]

        tokenized_texts.append(token_ids)
        all_token_ids.extend(token_ids)

    # Step 2: 패딩
    tensor_list = []
    for token_ids in tokenized_texts:
        tensor_list.append(torch.tensor(token_ids, dtype=torch.long))

    tensor = pad_sequence(tensor_list, batch_first=True, padding_value=0)

    vocab_size = sp.vocab_size()

    print(f"\n✅ 토크나이제이션 완료!")
    print(f"   최종 텐서 shape: {tensor.shape}")
    print(f"   Vocab size: {vocab_size:,}")
    print(f"   고유 토큰 수: {len(set(all_token_ids)):,}")

    return tensor, vocab_size


# ================================================================================
# ✅ Cell 7: DataLoader 생성 함수
# ================================================================================
from torch.utils.data import TensorDataset, DataLoader, random_split

def create_dataloaders(train_input_ids, train_labels,
                       test_input_ids, test_labels,
                       batch_size=32, val_ratio=0.2):
    """
    Train/Val/Test DataLoader 생성

    Args:
        train_input_ids: 훈련 데이터 입력 ID 텐서
        train_labels: 훈련 레이블
        test_input_ids: 테스트 데이터 입력 ID 텐서
        test_labels: 테스트 레이블
        batch_size: 배치 크기
        val_ratio: validation 비율

    Returns:
        train_loader, val_loader, test_loader
    """
    # Train 데이터에서 train/val split
    train_dataset = TensorDataset(train_input_ids, train_labels)
    val_size = int(len(train_dataset) * val_ratio)
    train_size = len(train_dataset) - val_size
    train_split, val_split = random_split(train_dataset, [train_size, val_size])

    # DataLoader 생성
    train_loader = DataLoader(train_split, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_split, batch_size=batch_size, shuffle=False)

    # Test DataLoader
    test_dataset = TensorDataset(test_input_ids, test_labels)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    print(f"✅ DataLoader 생성 완료!")
    print(f"   Train samples: {train_size:,} → {len(train_loader)} batches")
    print(f"   Val samples: {val_size:,} → {len(val_loader)} batches")
    print(f"   Test samples: {len(test_dataset):,} → {len(test_loader)} batches")
    print(f"   Batch size: {batch_size}")

    return train_loader, val_loader, test_loader


# 데이터 준비
corpus = train_processed['document'].tolist()
labels = torch.tensor(train_processed['label'].values, dtype=torch.long)

print(f"\n📊 원본 데이터:")
print(f"   Corpus 크기: {len(corpus):,}")
print(f"   Labels 크기: {len(labels):,}\n")

# ⭐ OKT + SentencePiece 토크나이징
train_input_ids, VOCAB_SIZE = okt_sentencepiece_tokenize(okt, sp, corpus, max_length=130)
print()

# Test 데이터도 처리
test_corpus = test_processed['document'].tolist()
test_labels = torch.tensor(test_processed['label'].values, dtype=torch.long)

test_input_ids, _ = okt_sentencepiece_tokenize(okt, sp, test_corpus, max_length=130)
print()

# DataLoader 생성
train_loader, val_loader, test_loader = create_dataloaders(
    train_input_ids,
    labels,
    test_input_ids,
    test_labels,
    batch_size=32,
    val_ratio=0.2
)

# 확인
print("=" * 60)
print("📊 DataLoader 샘플 확인")
print("=" * 60)

for batch in train_loader:
    x, y = batch
    print(f"Train - 입력 텐서 shape: {x.shape}")
    print(f"Train - 레이블 shape: {y.shape}")
    break

for batch in val_loader:
    x, y = batch
    print(f"Val - 입력 텐서 shape: {x.shape}")
    print(f"Val - 레이블 shape: {y.shape}")
    break

for batch in test_loader:
    x, y = batch
    print(f"Test - 입력 텐서 shape: {x.shape}")
    print(f"Test - 레이블 shape: {y.shape}")
    break


# ================================================================================
# ✅ Cell 8: 하이퍼파라미터 정의
# ================================================================================
import torch.nn as nn
import torch.nn.functional as F

# 공통 하이퍼파라미터 정의
EMBEDDING_DIM = 100                # 임베딩 벡터의 차원
HIDDEN_DIM = 128                   # LSTM의 은닉 상태 차원
OUTPUT_DIM = 1                     # 출력 차원 (긍정=1, 부정=0 -> 1개)
N_LAYERS = 2                       # LSTM 레이어 개수
BIDIRECTIONAL = True               # 양방향 RNN/LSTM 여부
DROPOUT_RATE = 0.2                 # 드롭아웃 비율
PAD_IDX = 0                        # 패딩 인덱스 (0)

print(f"📊 모델 하이퍼파라미터:")
print(f"   VOCAB_SIZE: {VOCAB_SIZE:,} (SentencePiece 동적 설정)")


# ================================================================================
# ✅ Cell 9: LSTM 모델
# ================================================================================
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, pad_idx):
        super().__init__()

        # 1. 임베딩 레이어
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

        # 2. LSTM 레이어
        self.lstm = nn.LSTM(input_size=embedding_dim,
                           hidden_size=hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           batch_first=True,
                           dropout=dropout)

        # 3. FC 레이어
        fc_input_dim = hidden_dim * 2
        self.fc = nn.Linear(fc_input_dim, output_dim)

        # 4. 드롭아웃
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        embedded = self.embedding(text)

        # 2. LSTM
        _output, (hidden, cell) = self.lstm(embedded)

        # 3. 마지막 레이어의 은닉 상태 결합 (양방향 처리)
        if self.lstm.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        else:
            hidden = self.dropout(hidden[-1,:,:])

        # 4. FC 레이어 통과
        prediction = self.fc(hidden)

        return prediction.squeeze(1)


# ================================================================================
# ✅ Cell 10: 훈련 함수
# ================================================================================
import torch
import torch.optim as optim
import time

# 0. GPU 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 1. 헬퍼 함수 정의
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# 2. 훈련 함수 정의
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for texts, labels in iterator:
        texts = texts.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        predictions = model(texts)
        loss = criterion(predictions, labels.float())
        acc = binary_accuracy(predictions, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# 3. 평가 함수 정의
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for texts, labels in iterator:
            texts = texts.to(device)
            labels = labels.to(device)
            predictions = model(texts)
            loss = criterion(predictions, labels.float())
            acc = binary_accuracy(predictions, labels)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


# ================================================================================
# ✅ Cell 11: 모델 훈련
# ================================================================================
# 모델 설정
lstm_model = LSTMModel(
    VOCAB_SIZE,
    EMBEDDING_DIM,
    HIDDEN_DIM,
    OUTPUT_DIM,
    N_LAYERS,
    BIDIRECTIONAL,
    DROPOUT_RATE,
    PAD_IDX
).to(device)

save_path = 'best_model_lstm_okt.pt'
N_EPOCHS = 20
patience = 5
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, lstm_model.parameters()), lr=0.0001)

# Early stopping 변수
best_valid_loss = float('inf')
patience_counter = 0

train_losses, train_accs, valid_losses, valid_accs = [], [], [], []

print(f"\n{'='*60}")
print(f"--- LSTM Model Training starts (OKT + SentencePiece) ---")
print(f"{'='*60}\n")

# 학습 루프
for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss, train_acc = train(lstm_model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(lstm_model, val_loader, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

    # 기록
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    valid_losses.append(valid_loss)
    valid_accs.append(valid_acc)

    # Early Stopping
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(lstm_model.state_dict(), save_path)
        patience_counter = 0
        print(f'\t>> Validation loss improved ({best_valid_loss:.3f}). Model saved.')
    else:
        patience_counter += 1
        print(f'\t>> Validation loss did not improve. Counter: {patience_counter}/{patience}')
        if patience_counter >= patience:
            print(f'--- Early stopping triggered after {epoch+1} epochs ---')
            break

# 테스트 평가
print(f"\n--- Loading best LSTM model for test evaluation ---")
lstm_model.load_state_dict(torch.load(save_path))
test_loss, test_acc = evaluate(lstm_model, test_loader, criterion)

print(f"\n--- LSTM Model Test Results (Best Model) ---")
print(f'\tTest Loss: {test_loss:.3f}')
print(f'\tTest Acc:  {test_acc*100:.2f}%')

# ================================================================================
# 🎉 코드 끝 (OKT + SentencePiece 버전)
# ================================================================================

✅ 데이터 로드 완료
Train shape: (150000, 3)
Test shape: (50000, 3)

전처리 결과 샘플:
         id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
데이터 전처리 시작

📊 Train 데이터 전처리:
전처리 전 데이터 크기: 150000
결측치 제거 후: 149995
빈 문자열 제거 후: 149607
중복 제거 후: 144478

📊 Test 데이터 전처리:
전처리 전 데이터 크기: 50000
결측치 제거 후: 49997
빈 문자열 제거 후: 49843
중복 제거 후: 48700

✅ 전처리 완료!
Train 데이터: 144478개
Test 데이터: 48700개

전처리 결과 샘플:
         id                                           document  label
0   9976970                                아 더빙 . 진짜 짜증나네요 목소리      0
1   3819312                   흠 . 포스터보고 초딩영화줄 . 오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다

OKT morphs: 100%|██████████| 144478/144478 [04:24<00:00, 545.31it/s]


✅ 형태소 분석 완료: /tmp/morphs_corpus_okt.txt

🔄 Step 2: SentencePiece 모델 학습 중...
✅ SentencePiece 모델 학습 완료
   Model: /tmp/sentencepiece_okt.model
   Vocab size: 8,000

✅ SentencePiece 모델 로드 완료
   Vocab size: 8000

OKT + SentencePiece 토크나이제이션 테스트

원본: 이 영화 정말 재미있었어
형태소: 이 영화 정말 재미있었어
인코딩: [7, 5, 17, 3089, 71]
디코딩: 이 영화 정말 재미있었어

원본: 최악의 영화였다
형태소: 최악 의 영화 였다
인코딩: [169, 76, 5, 415]
디코딩: 최악 의 영화 였다

원본: 배우들의 연기가 훌륭했다
형태소: 배우 들 의 연기 가 훌륭했다
인코딩: [63, 11, 76, 31, 25, 729, 635]
디코딩: 배우 들 의 연기 가 훌륭했다

📊 원본 데이터:
   Corpus 크기: 144,478
   Labels 크기: 144,478

🔄 OKT 형태소 분석 + SentencePiece 토크나이제이션 진행 중...


OKT + SentencePiece: 100%|██████████| 144478/144478 [06:51<00:00, 351.49it/s]



✅ 토크나이제이션 완료!
   최종 텐서 shape: torch.Size([144478, 111])
   Vocab size: 8,000
   고유 토큰 수: 7,989

🔄 OKT 형태소 분석 + SentencePiece 토크나이제이션 진행 중...


OKT + SentencePiece: 100%|██████████| 48700/48700 [02:09<00:00, 376.87it/s]



✅ 토크나이제이션 완료!
   최종 텐서 shape: torch.Size([48700, 110])
   Vocab size: 8,000
   고유 토큰 수: 7,355

✅ DataLoader 생성 완료!
   Train samples: 115,583 → 3612 batches
   Val samples: 28,895 → 903 batches
   Test samples: 48,700 → 1522 batches
   Batch size: 32
📊 DataLoader 샘플 확인
Train - 입력 텐서 shape: torch.Size([32, 111])
Train - 레이블 shape: torch.Size([32])
Val - 입력 텐서 shape: torch.Size([32, 111])
Val - 레이블 shape: torch.Size([32])
Test - 입력 텐서 shape: torch.Size([32, 110])
Test - 레이블 shape: torch.Size([32])
📊 모델 하이퍼파라미터:
   VOCAB_SIZE: 8,000 (SentencePiece 동적 설정)
Using device: cuda

--- LSTM Model Training starts (OKT + SentencePiece) ---

Epoch: 01 | Time: 0m 32s
	Train Loss: 0.538 | Train Acc: 71.77%
	 Val. Loss: 0.453 |  Val. Acc: 78.59%
	>> Validation loss improved (0.453). Model saved.
Epoch: 02 | Time: 0m 31s
	Train Loss: 0.426 | Train Acc: 80.36%
	 Val. Loss: 0.407 |  Val. Acc: 81.10%
	>> Validation loss improved (0.407). Model saved.
Epoch: 03 | Time: 0m 32s
	Train Loss: 0.382 | Train Acc: